In [1]:
# Library
import pandas as pd
import numpy as np
# import seaborn as sns
from tqdm import tqdm
import time
# import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime
from itertools import permutations
import pickle
import os
import gc
import re
import h5py

PATH = "e:/Financial_Data/"

In [17]:
p = PATH + 'Polygon/delisted_1d'
all_files = os.listdir(p)  # 현재 디렉토리의 모든 파일과 폴더 이름

# 파일 이름만 필터링
file_names = [f for f in all_files if os.path.isfile(os.path.join(p, f))]

In [10]:
df = pd.read_feather(PATH+"Polygon/delisted_1m/9366-9377.feather")

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078364 entries, 1721379060 to 1734013320
Data columns (total 70 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   (PSIG, open)          13764 non-null   float64
 1   (PSIG, high)          13764 non-null   float64
 2   (PSIG, low)           13764 non-null   float64
 3   (PSIG, close)         13764 non-null   float64
 4   (PSIG, volume)        13764 non-null   float64
 5   (PSIG, vwap)          13764 non-null   float64
 6   (PSIG, transactions)  13764 non-null   float64
 7   (PSMT, open)          389427 non-null  float64
 8   (PSMT, high)          389427 non-null  float64
 9   (PSMT, low)           389427 non-null  float64
 10  (PSMT, close)         389427 non-null  float64
 11  (PSMT, volume)        389427 non-null  float64
 12  (PSMT, vwap)          389427 non-null  float64
 13  (PSMT, transactions)  389427 non-null  float64
 14  (PSNL, open)          361512 non-null  floa

In [11]:
df.columns.names = ['ticker', 'ohlcv']
df_transformed = (
    df.stack(level='ticker')               # ticker 레벨을 인덱스로 내림
    .swaplevel(i=0, j=1)                   # 인덱스 레벨 순서를 ticker, timestamp로 변경
    .sort_index(level=['ticker', 'timestamp'])  # 인덱스를 정렬
)
df_transformed

ohlcv                 open     high      low    close  volume     vwap  \
ticker timestamp                                                         
CNHX   1445368560  30.7300  30.7300  30.7300  30.7300   100.0  30.7300   
       1445442420  29.3900  29.3900  29.3900  29.3900   100.0  29.3900   
       1445626140  30.9995  30.9995  30.9995  30.9995   175.0  30.9995   
       1446228780  29.8000  29.8000  29.8000  29.8000   210.0  29.8000   
       1447961280  31.9200  31.9200  31.9200  31.9200  1000.0  31.9200   
...                    ...      ...      ...      ...     ...      ...   
CSH    1681224840  25.5499  25.5499  25.5499  25.5499   200.0  25.5499   
       1681225020  25.5500  25.5500  25.5500  25.5500   200.0  25.5500   
       1681225860  25.5499  25.5499  25.5499  25.5499   400.0  25.5499   
       1681226640  25.5500  25.5500  25.5500  25.5500   200.0  25.5500   
       1681232880  25.2500  25.2500  25.1787  25.1787   455.0  25.2187   

ohlcv              transactions  
ticker timestamp                 
CNHX   1445368560           1.0  
       1445442420           1.0  
       1445626140           1.0  
       1446228780           1.0  
       1447961280          10.0  
...                         ...  
CSH    1681224840           1.0  
       1681225020           1.0  
       1681225860           1.0  
       1681226640           1.0  
       1681232880           3.0  

[523874 rows x 7 columns]

In [18]:
for f in tqdm(file_names[:]):
  df = pd.read_feather(PATH+"Polygon/delisted_1d/"+f)
  
  df.columns.names = ['ticker', 'ohlcv']
  df_transformed = (
      df.stack(level='ticker')               # ticker 레벨을 인덱스로 내림
      .swaplevel(i=0, j=1)                   # 인덱스 레벨 순서를 ticker, timestamp로 변경
      .sort_index(level=['ticker', 'timestamp'])  # 인덱스를 정렬
  )
  
  df_transformed = df_transformed.astype(dtype='float64')
  
  for ticker in df_transformed.index.get_level_values(level=0).unique():
    with pd.HDFStore(PATH+'data.h5', mode='a', complevel=5, complib='zlib') as store:
      mod_ticker = re.sub(r'\W', '_', ticker)
      store.put(f"chart/delisted_1d/{mod_ticker}", df_transformed.loc[ticker], format='table')

  0%|          | 0/21 [00:00<?, ?it/s]C:\Users\naniri\AppData\Local\Temp\ipykernel_5724\3123658841.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack(level='ticker')               # ticker 레벨을 인덱스로 내림
  5%|▍         | 1/21 [00:09<03:13,  9.67s/it]C:\Users\naniri\AppData\Local\Temp\ipykernel_5724\3123658841.py:6: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack(level='ticker')               # ticker 레벨을 인덱스로 내림
 10%|▉         | 2/21 [00:22<03:34, 11.31s/it]C:\Users\naniri\AppData\Local\Temp\ipykernel_5724\3123658841.py:6: FutureWarning: The previous imple

In [20]:
with h5py.File(PATH+'data.h5', mode='r') as store:
  ks = store['chart/delisted_1d'].keys()
  print(len(ks))

7854


In [3]:
gc.collect()

2041

In [16]:
pd.read_hdf(PATH+'data.h5', key=f'chart/inlisted_1d/AAPL').info()

<class 'pandas.core.frame.DataFrame'>
Index: 2505 entries, 1420174800 to 1734066000
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          2505 non-null   float64
 1   high          2505 non-null   float64
 2   low           2505 non-null   float64
 3   close         2505 non-null   float64
 4   volume        2505 non-null   float64
 5   vwap          2505 non-null   float64
 6   transactions  2505 non-null   float64
dtypes: float64(7)
memory usage: 156.6 KB
